In [67]:
import requests
from bs4 import BeautifulSoup
import time

iphone_url = "https://www.ebay.co.uk/sch/i.html?_from=R40&_trksid=p4432023.m570.l1313&_nkw=apple+watch&_sacat=0"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
urls = list()

for i in range(200):
    print(i)
    try:
        search_url = iphone_url + str(i+1)

        response = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        temp = [a["href"] for a in soup.find_all("a", class_="s-item__link")]

        urls = urls + temp
    except:
        break
    time.sleep(2)

    # Filter out non-product links
urls = [link for link in urls if "ebay.co.uk/itm/" in link]
urls = list(set(urls))
urls


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


['https://www.ebay.co.uk/itm/404487626451?hash=item5e2d575ed3:g:u~4AAOSwY7hlAE9k&amdata=enc%3AAQAIAAAA4EbNr3sCI8M4%2BXtOlaxd%2BA70F9ptwxwUARplgi69xuXEixR5QtsBHoQSAeCHK9JL1bzPJVRqZ3whbjNzC3aE2sx%2BEaZEyG5qGW1LYsRalXN1PbQUfhQM%2BaL3bOXhjto90j2j8aloY8AlelvL4iU6pMzoD%2B%2Bod3nXyjHQx%2F32eB872p3NTHbYLn2TEXinlPnNZSxTgYaMYMCcQ3jrSVklIH08dzh9Md4NQnMRWb5YXu7Zz9bRXk9j5Lrc%2F0zJpBOYPXMo%2Bk1o2GhCj3txA5P24tamQV67xZ5WYxbA%2F8bBUq%2Be%7Ctkp%3ABFBMmqOI8fFi',
 'https://www.ebay.co.uk/itm/202653524441?hash=item2f2f1761d9:g:1swAAOSwvQdcsyVC&amdata=enc%3AAQAIAAAAwIlBelMYWt1h9vd9oNx4K%2Fpn8GIgmRNZ3aO8rgMHjNtLLh%2BJQhxAIlCP9z9MSmNu87YAQ4O4E5GbMocaY%2FRE0p%2FOuWDbnMZo%2FDMk1v2jn03GYP28BbGOZkKUykJcEQ7Q2nf8Jc%2B5xXECUmd7ymDb3Cn1FLxYGAIy0mwnr%2F8QBUcWbm1NueLXrYdfweWXjAueay9HeiGmMmxlICuNTq8WldhGB958uVkIDqzd7DMnx5oHEmIt5rwljc2O539Hj%2BqXpw%3D%3D%7Ctkp%3ABk9SR_7W6PDxYg',
 'https://www.ebay.co.uk/itm/144924325313?hash=item21be29b5c1:g:yhMAAOSwsLRkka5p&amdata=enc%3AAQAIAAAA4F1Dg2I0fIpyg9vba1K%2FbvJ7N2Lmd3VAb3e8IiOC

In [71]:
import pandas as pd

# Initialize an empty list to store the results
#results = []

for url in range(898,len(urls)):
    print(url)
    # Send a GET request to the URL
    response = requests.get(urls[url], headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the elements
    try:
        product_title_element = soup.find('span', {'class': 'ux-textspans ux-textspans--BOLD'})
    except:
        continue
    price_element = soup.find('div', {'data-testid': 'x-price-primary'})
    reviews_element = soup.find('div', {'class': 'x-review-details__allreviews'})
    
    # Get the text from the elements
    try:
        product_title = product_title_element.get_text(strip=True)
    except:
        continue
    price = price_element.find(class_="ux-textspans").text.strip()
    try:
        reviews_link = reviews_element.find('a', {'data-testid': 'ux-action'})['href']
    except:
        continue
    reviews_count = int(reviews_element.find('a', {'data-testid': 'ux-action'}).text.strip().split()[2])
    if reviews_count % 10 == 0:
        pages_count = int(reviews_count/10)
    else:
        pages_count = int(reviews_count/10)+1
    pages_list = list()
    
    for i in range(pages_count):
        page = reviews_link + "&pgn=" + str(i+1)
        response = requests.get(page, headers=headers)
        
        html_content = response.content

        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all the review elements
        review_elements = soup.find_all(class_="ebay-review-section")
        

        # Loop through the review elements and extract the text
        for review_element in review_elements:
            try:
                review_text = review_element.find(class_="review-item-content").get_text(strip=True, separator='\n')
                
            except:
                review_text = ""
                pass
            try:
                review_title = review_element.find(class_="review-item-title").get_text(strip=True, separator='\n')
            except:
                review_title = ""
                pass
            try:
                review_star = review_element.find(class_="star-rating")['aria-label']
            except:
                review_star = ""
            try:
                review_author = review_element.find(class_="review-item-author").get_text(strip=True, separator='\n')
            except:
                review_author = ""
                pass
            try:
                review_date = review_element.find(class_="review-item-date").get_text(strip=True, separator='\n')
            except:
                review_date = ""
                pass
            try:
                review_verifiedPurchase = review_element.find_all(class_="rvw-val")[0].get_text(strip=True, separator='\n')
            except:
                review_verifiedPurchase = ""
                pass
            try:
                condition = review_element.find_all(class_="rvw-val")[1].get_text(strip=True, separator='\n')
            except:
                condition = ""
                pass
            try:
                seller = review_element.find_all(class_="rvw-val")[2].get_text(strip=True, separator='\n')
            except:
                seller = ""
                pass
            results.append({'Search': 'Apple Watch', 'Product Title': product_title, 'Condition': condition, 'Price': price, 'Seller': seller, 
                            'Review': review_text, 'Review - Title': review_title, 'Review - Star': review_star,
                            'Review - Author': review_author, 'Review - Date': review_date,  
                            'Review - Verified Purchase': review_verifiedPurchase})
            df = pd.DataFrame(results)
            df.to_csv('ebay_comments.csv', index=False)
        time.sleep(2)
        
df


898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005


KeyboardInterrupt: 

In [72]:
#df = pd.DataFrame(results)
df = df.drop_duplicates()
df.to_csv('ebay_comments.csv', index=False)
df

,Search,Product Title,Condition,Price,Seller,Review,Review - Title,Review - Star,Review - Author,Review - Date,Review - Verified Purchase
0,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,musicmagpie,£634.99,,Probably as I had hoped for as the phone is in...,Great but heavy.,4 stars,stickyprobs,"28 Jun, 2023",Yes
1,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,new,£634.99,ssa2234ssa,"Fantastic,new and sealed as described. It come...",Fantastic,5 stars,helwa1078-voxze...,"29 Sep, 2022",Yes
2,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,pre-owned,£634.99,phones2gadgets,It’s a good phone.. Had a new 13 model but fo...,Will do for me for a few years.. hopefully..,5 stars,paulprior249,"20 Jun, 2023",Yes
3,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,phonesdirect_com,£634.99,,Perfect quality as if its new and it has 100% ...,Goood,5 stars,88s275,"16 Aug, 2023",Yes
4,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,new,£634.99,sell-phones,I love it fantastic phone,Perfect 👍,5 stars,sarolga,"04 Jun, 2022",Yes
...,...,...,...,...,...,...,...,...,...,...,...
81237,Apple Watch,Apple Watch SE 44mm Space Grey Aluminum Case w...,pre-owned,£65.00,pk76r-81,helpful to keep fit.,nice watch.,5 stars,sgh-6059,"13 Feb, 2022",Yes
81238,Apple Watch,Apple Watch SE 44mm Space Grey Aluminum Case w...,pre-owned,£65.00,gadget_gogo,Great watch all round.,Apple Watch se,5 stars,bustercat123,"12 Feb, 2022",Yes
81239,Apple Watch,Apple Watch SE 44mm Space Grey Aluminum Case w...,pre-owned,£65.00,lapper365,Working perfectly fine,Awesome,5 stars,am230299,"08 Nov, 2022",Yes
81240,Apple Watch,Apple Watch SE 44mm Space Grey Aluminum Case w...,pre-owned,£65.00,7keating,Perfect with IPhone.,Perfect with IPhone.,5 stars,hifi_for_music,"23 Jun, 2021",Yes


In [73]:
df.Search.value_counts()

Iphone         51903
AirPods         5231
iPad            2354
Macbook         1331
Apple Watch     1128
Name: Search, dtype: int64

In [ ]:
Mac
iPhone
iPad
AirPods
Watch
